**웹캠**

In [9]:
import shutil
from keras.models import load_model
from keras.layers import DepthwiseConv2D
import cv2  
import numpy as np
import tensorflow as tf

from PIL import Image
import os

In [1]:
model = load_model("C:/Users/youngjin/Desktop/cabdi2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

class_names = open("C:/Users/youngjin/Desktop/cabdi2024/labels.txt", "r").readlines()


camera = cv2.VideoCapture(0)

while True:
    ret, image = camera.read()

    if not ret:
        print("Failed to grab camera image")
        break


    if image is None:
        print("Empty image")
        continue

    resized_image = cv2.resize(image, (224, 224))


    cv2.imshow("Webcam Image", resized_image)


    image = np.asarray(resized_image, dtype=np.float32)
    image = np.expand_dims(image, axis=0)


    image = image / 255.0


    prediction = model.predict(image)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]


    print("Class:", class_name[2:], end="")
    print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")


    keyboard_input = cv2.waitKey(1)


    if keyboard_input == 27:
        break

camera.release()
cv2.destroyAllWindows()




NameError: name 'load_model' is not defined

url

In [1]:
import cv2
import numpy as np
from keras.models import load_model
from keras.layers import DepthwiseConv2D
import requests
import json
from bs4 import BeautifulSoup

def food_info(name, ingredients):
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else:
        print("HTTP 응답 오류:", response.status_code)
        return

    food_list = soup.find_all(attrs={'class': 'common_sp_link'})
    if not food_list:
        print("식재료에 맞는 레시피가 없습니다.")
        return

    recipes = []

    for food in food_list:
        food_id = food['href'].split('/')[-1]
        new_url = f'https://www.10000recipe.com/recipe/{food_id}'
        new_response = requests.get(new_url)
        if new_response.status_code == 200:
            html = new_response.text
            soup = BeautifulSoup(html, 'html.parser')
        else:
            print("HTTP 응답 오류:", new_response.status_code)
            continue

        food_info = soup.find(attrs={'type': 'application/ld+json'})
        if food_info is not None:
            result = json.loads(food_info.text)
            recipe_ingredients = result.get('recipeIngredient', [])
            recipe = [step['text'] for step in result.get('recipeInstructions', [])]
            for i in range(len(recipe)):
                recipe[i] = f'{i + 1}. ' + recipe[i]

            view_count_element = soup.find(attrs={'class': 'view_count'})
            view_count = int(view_count_element.text.replace(',', '')) if view_count_element else 0

            print(f"레시피: {soup.title.string}, 조회수: {view_count}")

            recipes.append({
                'title': soup.title.string,
                'ingredients': recipe_ingredients,
                'recipe': recipe,
                'view_count': view_count
            })

    if recipes:
        best_recipe = max(recipes, key=lambda x: x['view_count'])

        # 추가적인 식재료 확인
        if ingredients:
            print(f"추가적인 식재료: {', '.join(ingredients)}")
            best_recipe['ingredients'].extend(ingredients)

        res = {
            'name': best_recipe['title'],
            'ingredients': ', '.join(best_recipe['ingredients']),
            'recipe': best_recipe['recipe']
        }
        return res
    else:
        print("적합한 레시피를 찾지 못했습니다.")
        return

# 모델과 라벨 로드
model = load_model("C:/Users/youngjin/Desktop/cabdi2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

class_names = open("C:/Users/youngjin/Desktop/cabdi2024/labels.txt", "r").readlines()

# 웹캠 초기화
camera = cv2.VideoCapture(0)
is_recognizing = True
additional_ingredients = []

while True:
    ret, image = camera.read()
    if not ret:
        print("웹캠 이미지 캡처 실패")
        break

    resized_image = cv2.resize(image, (224, 224))
    cv2.imshow("Webcam Image", resized_image)

    if is_recognizing:
        image = np.asarray(resized_image, dtype=np.float32)
        image = np.expand_dims(image, axis=0)
        image = image / 255.0

        prediction = model.predict(image)
        index = np.argmax(prediction)
        class_name = class_names[index].strip()
        confidence_score = prediction[0][index]

        print("Class:", class_name[2:], "Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")

        if confidence_score > 0.5:  # 낮은 신뢰도 예측을 피하기 위한 임계값 설정
            print(f"인식된 음식: {class_name[2:]}. 맞습니까? (y/n)")
            while True:
                keyboard_input = cv2.waitKey(1)
                if keyboard_input == ord('y'):
                    food_info_result = food_info(class_name[2:], additional_ingredients)
                    if food_info_result:
                        print("Food Name:", food_info_result["name"])
                        print("Ingredients:", food_info_result["ingredients"])
                        print("Recipe:")
                        for step in food_info_result['recipe']:
                            print(step)
                    is_recognizing = False  # 인식을 중단하고 다음 단계로 넘어감
                    additional_ingredients = []  # 추가적인 식재료 초기화
                    break
                elif keyboard_input == ord('n'):
                    print("레시피를 추천합니다.")
                    break
                elif keyboard_input == ord('a'):
                    print("레시피에 추가할 식재료가 있습니까? (y/n)")
                    additional_input = cv2.waitKey(0)
                    if additional_input == ord('y'):
                        ingredient_input = input("추가할 식재료를 입력하세요: ")
                        additional_ingredients.append(ingredient_input)
                    elif additional_input == ord('n'):
                        print("추가적인 식재료 입력을 종료합니다.")
                        break
                    else:
                        print("잘못된 입력입니다.")
                elif keyboard_input == 27:  # ESC 키를 누르면 종료
                    is_recognizing = False
                    break

    if not is_recognizing:
        break

camera.release()
cv2.destroyAllWindows()





1/1 [==============================] - 1s 1s/step
Class: nasi putih Confidence Score: 100 %
인식된 음식: nasi putih. 맞습니까? (y/n)
식재료에 맞는 레시피가 없습니다.


In [1]:
import cv2
import numpy as np
from keras.models import load_model
from keras.layers import DepthwiseConv2D
import requests
import json
from bs4 import BeautifulSoup

def food_info(name):
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else:
        print("HTTP 응답 오류:", response.status_code)
        return

    food_list = soup.find_all(attrs={'class': 'common_sp_link'})
    if not food_list:
        print("식재료에 맞는 레시피가 없습니다.")
        return

    recipes = []

    for food in food_list:
        food_id = food['href'].split('/')[-1]
        new_url = f'https://www.10000recipe.com/recipe/{food_id}'
        new_response = requests.get(new_url)
        if new_response.status_code == 200:
            html = new_response.text
            soup = BeautifulSoup(html, 'html.parser')
        else:
            print("HTTP 응답 오류:", new_response.status_code)
            continue

        food_info = soup.find(attrs={'type': 'application/ld+json'})
        if food_info is not None:
            result = json.loads(food_info.text)
            ingredients = result.get('recipeIngredient', [])
            recipe = [step['text'] for step in result.get('recipeInstructions', [])]
            for i in range(len(recipe)):
                recipe[i] = f'{i + 1}. ' + recipe[i]

            view_count_element = soup.find(attrs={'class': 'view_count'})
            view_count = int(view_count_element.text.replace(',', '')) if view_count_element else 0

            print(f"레시피: {soup.title.string}, 조회수: {view_count}")

            recipes.append({
                'title': soup.title.string,
                'ingredients': ingredients,
                'recipe': recipe,
                'view_count': view_count
            })

    if recipes:
        best_recipe = max(recipes, key=lambda x: x['view_count'])

        res = {
            'name': best_recipe['title'],
            'ingredients': ', '.join(best_recipe['ingredients']),
            'recipe': best_recipe['recipe']
        }
        return res
    else:
        print("적합한 레시피를 찾지 못했습니다.")
        return

# 모델과 라벨 로드
model = load_model("C:/Users/youngjin/Desktop/cabdi2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

class_names = open("C:/Users/youngjin/Desktop/cabdi2024/labels.txt", "r").readlines()

# 웹캠 초기화
camera = cv2.VideoCapture(0)
is_recognizing = True

while True:
    ret, image = camera.read()
    if not ret:
        print("웹캠 이미지 캡처 실패")
        break

    resized_image = cv2.resize(image, (224, 224))
    cv2.imshow("Webcam Image", resized_image)

    if is_recognizing:
        image = np.asarray(resized_image, dtype=np.float32)
        image = np.expand_dims(image, axis=0)
        image = image / 255.0

        prediction = model.predict(image)
        index = np.argmax(prediction)
        class_name = class_names[index].strip()
        confidence_score = prediction[0][index]

        print("Class:", class_name[2:], "Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")

        if confidence_score > 0.5:  # 낮은 신뢰도 예측을 피하기 위한 임계값 설정
            print(f"인식된 음식: {class_name[2:]}. 맞습니까? (y/n)")
            while True:
                keyboard_input = cv2.waitKey(1)
                if keyboard_input == ord('y'):
                    food_info_result = food_info(class_name[2:])
                    if food_info_result:
                        print("Food Name:", food_info_result["name"])
                        print("Ingredients:", food_info_result["ingredients"])
                        print("Recipe:")
                        for step in food_info_result['recipe']:
                            print(step)
                    is_recognizing = False  # 인식을 중단하고 다음 단계로 넘어감
                    break
                elif keyboard_input == ord('n'):
                    is_recognizing = True  # 인식을 계속 진행
                    break
                elif keyboard_input == 27:  # ESC 키를 누르면 종료
                    is_recognizing = False
                    break

    if not is_recognizing:
        break

camera.release()
cv2.destroyAllWindows()





웹캠 이미지 캡처 실패


In [4]:
import numpy as np
from tensorflow.keras.models import load_model
from keras.layers import DepthwiseConv2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import os


In [5]:
model = load_model("C:/Users/youngjin/Desktop/cabdi2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

class_names = open("C:/Users/youngjin/Desktop/cabdi2024/labels.txt", "r").readlines()
class_names = [name.strip() for name in class_names]  # 줄바꿈 문자 제거


In [8]:
test_dir = "C:/Users/youngjin/Desktop/cabdi2024/test_images"
image_size = (224, 224)  # 이미지 크기를 모델에 맞게 설정합니다.

# 이미지와 레이블을 로드합니다.
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = image.load_img(img_path, target_size=image_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        images.append(img_array)
        
        # 파일명에서 라벨 추출 (예: "cat.0.jpg" -> "cat")
        label = filename.split('.')[0]
        labels.append(label)
    return np.vstack(images), labels

test_images, test_labels = load_images_from_folder(test_dir)


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'C:/Users/youngjin/Desktop/cabdi2024/test_images'

In [ ]:
# 예측을 수행합니다.
predictions = model.predict(test_images)

# 예측된 클래스 인덱스를 얻습니다.
predicted_labels = [class_names[np.argmax(pred)] for pred in predictions]

# 정확도를 계산합니다.
correct_predictions = np.sum(np.array(predicted_labels) == np.array(test_labels))
accuracy = correct_predictions / len(test_labels)

print(f"테스트 정확도: {accuracy:.4f}")


**아두이노**

In [ ]:
#1
import shutil
from keras.models import load_model
from keras.layers import DepthwiseConv2D
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import serial
import time

def unzip_files(zip_path, extract_path):
    shutil.unpack_archive(zip_path, extract_path)

zip_path1 = 'C:/Users/youngjin/Desktop/캡디2024/Cheese.v1i.createml.zip'
zip_path2 = 'C:/Users/youngjin/Desktop/캡디2024/onion.v1i.createml.zip'
extract_path = 'C:/Users/youngjin/Desktop/캡디2024'

unzip_files(zip_path1, extract_path)
unzip_files(zip_path2, extract_path)

cheese = zip_path1
bread = zip_path2


# Load the model
model = load_model("C:/Users/youngjin/Desktop/캡디2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

# Load the labels
class_names = open("C:/Users/youngjin/Desktop/캡디2024/labels.txt", "r").readlines()

# Set up serial communication with Arduino
ser = serial.Serial('COM3', 9600)  # Replace 'COM3' with your Arduino's COM port
time.sleep(2)  # Wait for the serial connection to initialize

while True:
    # Read from Arduino
    if ser.in_waiting > 0:
        line = ser.readline().decode('utf-8').rstrip()
        print("Received:", line)
        
        # Assuming the Arduino sends the image data in a specific format
        # For example, if Arduino sends base64 encoded image data, you need to decode it
        try:
            # Decode the base64 string to bytes
            image_data = base64.b64decode(line)
            
            # Convert bytes to a PIL image
            image = Image.open(io.BytesIO(image_data))
            
            # Resize the image to (224, 224) pixels
            resized_image = image.resize((224, 224))
            
            # Convert the image to a numpy array
            image_array = np.asarray(resized_image, dtype=np.float32)
            image_array = np.expand_dims(image_array, axis=0)
            
            # Normalize the image array
            image_array = image_array / 255.0
            
            # Predict the model
            prediction = model.predict(image_array)
            index = np.argmax(prediction)
            class_name = class_names[index]
            confidence_score = prediction[0][index]

            # Print prediction and confidence score
            print("Class:", class_name[2:], end="")
            print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")
        
        except Exception as e:
            print("Error processing image:", e)

ser.close()


In [ ]:
#2
import shutil
from keras.models import load_model
from keras.layers import DepthwiseConv2D
import numpy as np
import tensorflow as tf
from PIL import Image
import io
import serial

def unzip_files(zip_path, extract_path):
    shutil.unpack_archive(zip_path, extract_path)

zip_path1 = 'C:/Users/youngjin/Desktop/캡디2024/Cheese.v1i.createml.zip'
zip_path2 = 'C:/Users/youngjin/Desktop/캡디2024/onion.v1i.createml.zip'
extract_path = 'C:/Users/youngjin/Desktop/캡디2024'

unzip_files(zip_path1, extract_path)
unzip_files(zip_path2, extract_path)

cheese = zip_path1
bread = zip_path2

# Load the model
model = load_model("C:/Users/youngjin/Desktop/캡디2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

# Load the labels
class_names = open("C:/Users/youngjin/Desktop/캡디2024/labels.txt", "r").readlines()

# Setup serial communication
ser = serial.Serial('COM3', 115200, timeout=10)  # Replace 'COM3' with your serial port

def predict_image(image):
    # Preprocess the image for the model
    image = np.asarray(image, dtype=np.float32)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0

    # Predict the image
    prediction = model.predict(image)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    return class_name.strip(), confidence_score

def main():
    while True:
        # Read the image size
        image_size_bytes = ser.read(4)
        if len(image_size_bytes) != 4:
            continue
        
        image_size = int.from_bytes(image_size_bytes, byteorder='little')

        # Read the image data
        image_data = ser.read(image_size)
        if len(image_data) != image_size:
            continue

        # Load the image
        image = Image.open(io.BytesIO(image_data))

        # Predict the image
        class_name, confidence_score = predict_image(image)

        # Print the prediction result
        print(f"Class: {class_name}, Confidence Score: {confidence_score * 100:.2f}%")

if __name__ == "__main__":
    main()


In [ ]:
#3
import shutil
from keras.models import load_model
from keras.layers import DepthwiseConv2D
import numpy as np
import tensorflow as tf
from PIL import Image
import os
import serial
import time

def unzip_files(zip_path, extract_path):
    shutil.unpack_archive(zip_path, extract_path)

zip_path1 = 'C:/Users/youngjin/Desktop/캡디2024/Cheese.v1i.createml.zip'
zip_path2 = 'C:/Users/youngjin/Desktop/캡디2024/onion.v1i.createml.zip'
extract_path = 'C:/Users/youngjin/Desktop/캡디2024'

unzip_files(zip_path1, extract_path)
unzip_files(zip_path2, extract_path)

cheese = zip_path1
bread = zip_path2


# Load the model
model = load_model("C:/Users/youngjin/Desktop/캡디2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

# Load the labels
class_names = open("C:/Users/youngjin/Desktop/캡디2024/labels.txt", "r").readlines()

# Set up serial communication with Arduino
ser = serial.Serial('COM3', 9600)  # Replace 'COM3' with your Arduino's COM port
time.sleep(2)  # Wait for the serial connection to initialize

while True:
    # Read from Arduino
    if ser.in_waiting > 0:
        line = ser.readline().decode('utf-8').rstrip()
        print("Received:", line)
        
        # Assuming the Arduino sends the image data in a specific format
        # For example, if Arduino sends base64 encoded image data, you need to decode it
        try:
            # Convert the image data string to a PIL image
            image = Image.open(io.BytesIO(line))
            
            # Resize the image to (224, 224) pixels
            resized_image = image.resize((224, 224))
            
            # Convert the image to a numpy array
            image_array = np.asarray(resized_image, dtype=np.float32)
            image_array = np.expand_dims(image_array, axis=0)
            
            # Normalize the image array
            image_array = image_array / 255.0
            
            # Predict the model
            prediction = model.predict(image_array)
            index = np.argmax(prediction)
            class_name = class_names[index]
            confidence_score = prediction[0][index]

            # Print prediction and confidence score
            print("Class:", class_name[2:], end="")
            print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")
        
        except Exception as e:
            print("Error processing image:", e)

ser.close()


In [ ]:
#4
import serial
import numpy as np
import io
from PIL import Image
from keras.models import load_model
import tensorflow as tf

# 시리얼 포트 초기화
ser = serial.Serial('COM3', 115200)  # 시리얼 포트와 속도를 맞춰줘야 함
time.sleep(2)  # 시리얼 연결이 초기화될 때까지 대기

# 모델 로드
model = load_model("C:/Users/youngjin/Desktop/캡디2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

# 레이블 로드
class_names = open("C:/Users/youngjin/Desktop/캡디2024/labels.txt", "r").readlines()

def process_image(image_data):
    # 바이트 데이터를 이미지로 변환
    image = Image.open(io.BytesIO(image_data))
    image = image.resize((224, 224))
    image_array = np.asarray(image, dtype=np.float32)
    image_array = np.expand_dims(image_array, axis=0)
    image_array = image_array / 255.0
    return image_array

while True:
    # 시리얼 데이터 수신
    if ser.in_waiting > 0:
        photo_data = ser.read(ser.in_waiting)
        
        # 이미지 데이터 처리
        try:
            processed_image = process_image(photo_data)
            
            # 모델 예측
            prediction = model.predict(processed_image)
            index = np.argmax(prediction)
            class_name = class_names[index]
            confidence_score = prediction[0][index]

            # 예측 결과 출력
            print(f"Class: {class_name[2:]}, Confidence Score: {str(np.round(confidence_score * 100))[:-2]}%")
        
        except Exception as e:
            print(f"Error processing image: {e}")

ser.close()


**url**

In [ ]:
import serial
import numpy as np
import io
from PIL import Image
from keras.models import load_model
import tensorflow as tf
import time

# 시리얼 포트 초기화
ser = serial.Serial('COM3', 115200)  # 시리얼 포트와 속도를 맞춰줘야 함
time.sleep(2)  # 시리얼 연결이 초기화될 때까지 대기

# 모델 로드
model = load_model("C:/Users/youngjin/Desktop/캡디2024/keras_model.h5", 
                   custom_objects={'DepthwiseConv2D': DepthwiseConv2D}, 
                   compile=False)

# 레이블 로드
class_names = open("C:/Users/youngjin/Desktop/캡디2024/labels.txt", "r").readlines()

def process_image(image_data):
    # 바이트 데이터를 이미지로 변환
    image = Image.open(io.BytesIO(image_data))
    image = image.resize((224, 224))
    image_array = np.asarray(image, dtype=np.float32)
    image_array = np.expand_dims(image_array, axis=0)
    image_array = image_array / 255.0
    return image_array

def predict_image():
    while True:
        # 시리얼 데이터 수신
        if ser.in_waiting > 0:
            photo_data = ser.read(ser.in_waiting)
            
            # 이미지 데이터 처리
            try:
                processed_image = process_image(photo_data)
                
                # 모델 예측
                prediction = model.predict(processed_image)
                index = np.argmax(prediction)
                class_name = class_names[index]
                confidence_score = prediction[0][index]

                # 예측 결과 출력 및 반환
                print(f"Class: {class_name[2:]}, Confidence Score: {str(np.round(confidence_score * 100))[:-2]}%")
                return class_name[2:]  # 예측된 클래스 이름 반환
            
            except Exception as e:
                print(f"Error processing image: {e}")

if __name__ == "__main__":
    predicted_class = predict_image()
    ser.close()


In [ ]:
import requests
import json
from bs4 import BeautifulSoup
from predict_image import predict_image  # 모델 예측 함수를 import

def food_info(name):
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}"
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else:
        print("HTTP response error:", response.status_code)
        return

    food_list = soup.find_all(attrs={'class':'common_sp_link'})
    if not food_list:
        print("식재료에 맞는 레시피가 없습니다.")
        return

    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)
    if new_response.status_code == 200:
        html = new_response.text
        soup = BeautifulSoup(html, 'html.parser')
    else:
        print("HTTP response error:", new_response.status_code)
        return

    food_info = soup.find(attrs={'type':'application/ld+json'})
    result = json.loads(food_info.text)
    ingredient = ','.join(result['recipeIngredient'])
    recipe = [result['recipeInstructions'][i]['text'] for i in range(len(result['recipeInstructions']))]
    for i in range(len(recipe)):
        recipe[i] = f'{i+1}. ' + recipe[i]

    title = soup.title.string

    res = {
        'name': title,
        'ingredients': ingredient,
        'recipe': recipe
    }

    return res

if __name__ == "__main__":
    predicted_ingredients = predict_image()  # 모델 예측 함수 호출
    if predicted_ingredients:
        ingredients_str = " ".join(predicted_ingredients)  # 예측된 식재료를 공백으로 구분된 문자열로 변환
        food_info_result = food_info(ingredients_str)
        if food_info_result:
            print("Food Name:", food_info_result['name'])
            print("Ingredients:", food_info_result['ingredients'])
            print("Recipe:")
            for step in food_info_result['recipe']:
                print(step)


부트캠프 문제풀이

In [10]:
fact = 1
num = 1
while num <= 10:
    fact = num * fact
    print(f"10! = {fact}")
    num += 1

10! = 1
10! = 2
10! = 6
10! = 24
10! = 120
10! = 720
10! = 5040
10! = 40320
10! = 362880
10! = 3628800


In [40]:
x= 0.393701
q= 6
print("-" *35)
print("cm"," "*q,"mm"," "*q,"m"," "*q,"inch")    
print("-" *35)


for cm in range(1,51):
    mm = cm * 10 
    m = round(cm * 0.01,2)
    inch = round(cm * x,2)
    print(f"{cm:<10}{mm:<10}{m:<10}{inch:<10}")
    

-----------------------------------
cm        mm        m        inch
-----------------------------------
1         10        0.01      0.39      
2         20        0.02      0.79      
3         30        0.03      1.18      
4         40        0.04      1.57      
5         50        0.05      1.97      
6         60        0.06      2.36      
7         70        0.07      2.76      
8         80        0.08      3.15      
9         90        0.09      3.54      
10        100       0.1       3.94      
11        110       0.11      4.33      
12        120       0.12      4.72      
13        130       0.13      5.12      
14        140       0.14      5.51      
15        150       0.15      5.91      
16        160       0.16      6.3       
17        170       0.17      6.69      
18        180       0.18      7.09      
19        190       0.19      7.48      
20        200       0.2       7.87      
21        210       0.21      8.27      
22        220       0.22      8.6